In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector

In [2]:
# 载入数据集
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)
# 运行次数
max_steps = 1001
# 图片数量
image_num = 3000
# 文件路径
DIR = 'G:/DeepLearning/noopy/tensorflow_demo_1/'

# 定义回话
sess = tf.Session()

# 载入图片
embedding = tf.Variable(tf.stack(mnist.test.images[:image_num]), trainable=False, name='embedding')

# 参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev) # 标准差
        tf.summary.scalar('max', tf.reduce_max(var)) # 最大值
        tf.summary.scalar('min', tf.reduce_min(var)) # 最小值
        tf.summary.histogram('histogram', var) # 直方图

# 命名空间
with tf.name_scope('input'):
    # 定义两个placeholder
    x = tf.placeholder(tf.float32, [None, 784], 'x-input')
    y = tf.placeholder(tf.float32, [None, 10], 'y-input')
    
# 显示图片
with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', image_shaped_input, 10)

with tf.name_scope('layer'):
    # 创建一个简单的神经网络
    with tf.name_scope('weight'):
        W = tf.Variable(tf.zeros([784, 10]), name='W')
        variable_summaries(W)
    with tf.name_scope('bias'):
        b = tf.Variable(tf.zeros([10]), name='b')
        variable_summaries(b)
    with tf.name_scope('wx_plus_b'):
        wx_plus_b = tf.matmul(x, W) + b
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(wx_plus_b)

# 交叉熵代价函数
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
    tf.summary.scalar('loss', loss)
    
# 优化器：使用梯度下降法
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
    
# 初始化变量
sess.run(tf.global_variables_initializer())

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        # 预测准确率
        correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(prediction, 1))
    with tf.name_scope('accuracy'):
        # 计算准确率
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy', accuracy)

# 产生metadata文件
if tf.gfile.Exists(DIR + 'projector/projector/metadata.tsv'):
    tf.gfile.DeleteRecursively(DIR + 'projector/projector/metadata.tsv')
with open(DIR + 'projector/projector/metadata.tsv', 'w') as f:
    labels = sess.run(tf.argmax(mnist.test.labels[:],1))
    for i in range(image_num):
        f.write(str(labels[i]) + '\n')

# 合并所有的summary
merged = tf.summary.merge_all()

projector_writer = tf.summary.FileWriter(DIR + 'projector/projector', sess.graph)
saver = tf.train.Saver()
config = projector.ProjectorConfig()
embed  = config.embeddings.add()
embed.tensor_name = embedding.name
embed.metadata_path = DIR + 'projector/projector/metadata.tsv'
embed.sprite.image_path = DIR + 'projector/data/mnist_10k_sprite.png'
embed.sprite.single_image_dim.extend([28, 28])
projector.visualize_embeddings(projector_writer, config)

for i in range(max_steps):
    # 每个批次100个样本
    batch_xs, batch_ys = mnist.train.next_batch(100)
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    summary,_ = sess.run([merged,train_step],feed_dict={x: batch_xs, y: batch_ys}, options=run_options, run_metadata=run_metadata)
    projector_writer.add_run_metadata(run_metadata, 'step%03d' % i)
    projector_writer.add_summary(summary, i)
    
    acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
    print('Iter' + str(i) + ',Test Accuracy= ' + str(acc))

saver.save(sess, DIR + 'projector/projector/a_model.ckpt', global_step = max_steps)
projector_writer.close()
sess.close()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Iter0,Test Accuracy= 0.4058
Iter1,Test Accuracy= 0.4222
Iter2,Test Accuracy= 0.5055
Iter3,Test Accuracy= 0.418

Iter195,Test Accuracy= 0.7991
Iter196,Test Accuracy= 0.8004
Iter197,Test Accuracy= 0.8009
Iter198,Test Accuracy= 0.8017
Iter199,Test Accuracy= 0.8018
Iter200,Test Accuracy= 0.8019
Iter201,Test Accuracy= 0.801
Iter202,Test Accuracy= 0.8021
Iter203,Test Accuracy= 0.8014
Iter204,Test Accuracy= 0.8016
Iter205,Test Accuracy= 0.8019
Iter206,Test Accuracy= 0.8025
Iter207,Test Accuracy= 0.8029
Iter208,Test Accuracy= 0.8043
Iter209,Test Accuracy= 0.8041
Iter210,Test Accuracy= 0.8033
Iter211,Test Accuracy= 0.8025
Iter212,Test Accuracy= 0.8035
Iter213,Test Accuracy= 0.803
Iter214,Test Accuracy= 0.8037
Iter215,Test Accuracy= 0.8055
Iter216,Test Accuracy= 0.804
Iter217,Test Accuracy= 0.8049
Iter218,Test Accuracy= 0.8054
Iter219,Test Accuracy= 0.8057
Iter220,Test Accuracy= 0.806
Iter221,Test Accuracy= 0.8056
Iter222,Test Accuracy= 0.8053
Iter223,Test Accuracy= 0.8057
Iter224,Test Accuracy= 0.8045
Iter225,Test Accuracy= 0.8055
Iter226,Test Accuracy= 0.8052
Iter227,Test Accuracy= 0.8048
Iter228,Test A

Iter479,Test Accuracy= 0.8226
Iter480,Test Accuracy= 0.8227
Iter481,Test Accuracy= 0.8225
Iter482,Test Accuracy= 0.8219
Iter483,Test Accuracy= 0.8222
Iter484,Test Accuracy= 0.8227
Iter485,Test Accuracy= 0.823
Iter486,Test Accuracy= 0.8232
Iter487,Test Accuracy= 0.8227
Iter488,Test Accuracy= 0.8233
Iter489,Test Accuracy= 0.8231
Iter490,Test Accuracy= 0.823
Iter491,Test Accuracy= 0.8217
Iter492,Test Accuracy= 0.8222
Iter493,Test Accuracy= 0.8225
Iter494,Test Accuracy= 0.8222
Iter495,Test Accuracy= 0.8229
Iter496,Test Accuracy= 0.8226
Iter497,Test Accuracy= 0.8228
Iter498,Test Accuracy= 0.8231
Iter499,Test Accuracy= 0.8218
Iter500,Test Accuracy= 0.823
Iter501,Test Accuracy= 0.8234
Iter502,Test Accuracy= 0.8233
Iter503,Test Accuracy= 0.8234
Iter504,Test Accuracy= 0.8243
Iter505,Test Accuracy= 0.8243
Iter506,Test Accuracy= 0.8237
Iter507,Test Accuracy= 0.8229
Iter508,Test Accuracy= 0.8229
Iter509,Test Accuracy= 0.8239
Iter510,Test Accuracy= 0.8234
Iter511,Test Accuracy= 0.8227
Iter512,Test 

Iter756,Test Accuracy= 0.8298
Iter757,Test Accuracy= 0.8292
Iter758,Test Accuracy= 0.8299
Iter759,Test Accuracy= 0.8297
Iter760,Test Accuracy= 0.8305
Iter761,Test Accuracy= 0.8307
Iter762,Test Accuracy= 0.8311
Iter763,Test Accuracy= 0.831
Iter764,Test Accuracy= 0.8308
Iter765,Test Accuracy= 0.8307
Iter766,Test Accuracy= 0.8312
Iter767,Test Accuracy= 0.8309
Iter768,Test Accuracy= 0.8306
Iter769,Test Accuracy= 0.8305
Iter770,Test Accuracy= 0.8313
Iter771,Test Accuracy= 0.8312
Iter772,Test Accuracy= 0.8303
Iter773,Test Accuracy= 0.8307
Iter774,Test Accuracy= 0.8308
Iter775,Test Accuracy= 0.8316
Iter776,Test Accuracy= 0.8308
Iter777,Test Accuracy= 0.8307
Iter778,Test Accuracy= 0.8312
Iter779,Test Accuracy= 0.8309
Iter780,Test Accuracy= 0.8316
Iter781,Test Accuracy= 0.8315
Iter782,Test Accuracy= 0.8313
Iter783,Test Accuracy= 0.8312
Iter784,Test Accuracy= 0.8316
Iter785,Test Accuracy= 0.8326
Iter786,Test Accuracy= 0.8334
Iter787,Test Accuracy= 0.8329
Iter788,Test Accuracy= 0.8328
Iter789,Tes